In [1]:
import os
os.chdir('../')
%pwd

'D:\\projects\\Project-WineQualityDataset'

In [45]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ij451r/Project-WineQualityDataset.mlflow" 
os.environ["MLFLOW_TRACKING_USERNAME"]="ij451r"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cedc8e6cef3cb8ab1d4b37a9e530a44cedd8a1bb"

In [8]:
MLFLOW_TRACKING_URI="https://dagshub.com/ij451r/Project-WineQualityDataset.mlflow" 
MLFLOW_TRACKING_USERNAME="ij451r"
MLFLOW_TRACKING_PASSWORD="cedc8e6cef3cb8ab1d4b37a9e530a44cedd8a1bb"

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    x_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [68]:
from WineQuality.constants import * 
from WineQuality.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH , schema_filepath = SCHEMA_FILE_PATH,):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params.ElasticNet,
            target_column = schema.name,
            mlflow_uri = MLFLOW_TRACKING_URI,
        )

        return model_evaluation_config
        

In [71]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import numpy as np
import joblib
from WineQuality.utils.common import save_json
from pathlib import Path


class ModelEvaluation:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    def log_into_mlflow(self):
        model = joblib.load(self.config.model_path)
        
        X_test = pd.read_csv(self.config.x_test_data_path)
        Y_test = pd.read_csv(self.config.y_test_data_path)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            qualities_pred = model.predict(X_test)
            signature = infer_signature(X_test, qualities_pred)
            (rmse, mae, r2) = self.eval_metrics(Y_test, qualities_pred)
            scores = {
                "rmse":rmse,
                "mae":mae,
                "r2":r2
            }
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            mlflow.sklearn.log_model(model, "model")
            # mlflow.sklearn.log_model(
            #     model,
            #     artifact_path = "sklearn-model",
            #     signature = signature,
            #     registered_model_name="sk-learn-elastic-net-model",
            # )
            

In [73]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-09-30 16:49:53,111: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-30 16:49:53,113: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-30 16:49:53,116: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-30 16:49:53,118: INFO: common: Directory Created: artifacts]
[2023-09-30 16:49:53,119: INFO: common: Directory Created: artifacts/model_evaluation]
[2023-09-30 16:49:54,219: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]
{'alpha': 0.5, 'l1_ratio': 0.01}
